In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import nltk
from konlpy.tag import Kkma

kor_tagger = Kkma()

In [2]:
# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
eng = nltk.word_tokenize("Hi, my name is Youngin. What's your name?")
print(eng)

['Hi', ',', 'my', 'name', 'is', 'Youngin', '.', 'What', "'s", 'your', 'name', '?']


In [4]:
kor = kor_tagger.morphs("안녕하세요! 저는 파이토치를 공부하는 중입니다!")
print(kor)

['안녕', '하', '세요', '!', '저', '는', '파이', '토치', '를', '공부', '하', '는', '중', '이', 'ㅂ니다', '!']


In [5]:
# dictionary for indexing
word2idx = {}
for token in kor:
    if word2idx.get(token) == None:
        word2idx[token] = len(word2idx) # 순차적으로 token에 index을 할당함
print(word2idx)

{'안녕': 0, '하': 1, '세요': 2, '!': 3, '저': 4, '는': 5, '파이': 6, '토치': 7, '를': 8, '공부': 9, '중': 10, '이': 11, 'ㅂ니다': 12}


In [6]:
# function to one-hot encoding
def one_hot_encoding(word, word2idx):
    tensor = torch.zeros(len(word2idx))
    index = word2idx[word]
    tensor[index] = 1. # 단어에 해당하는 index에만 1을 할당. 나머지는 모두 0
    
    return tensor

In [7]:
vec = one_hot_encoding("토치", word2idx)
print(vec)
print(vec.size())

tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.])
torch.Size([13])


In [8]:
vec = one_hot_encoding("파이", word2idx)
print(vec)
print(vec.size())

tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.])
torch.Size([13])


In [9]:
# 문장과 해당 문장의 카테고리를 나열한 데이터
train_data = [["배고프다 밥줘","FOOD"],
                    ["뭐 먹을만한거 없냐","FOOD"],
                    ["맛집 추천","FOOD"],
                    ["이 근처 맛있는 음식점 좀","FOOD"],
                    ["밥줘","FOOD"],
                    ["뭐 먹지?","FOOD"],
                    ["삼겹살 먹고싶어","FOOD"],
                    ["영화 보고싶다","MEDIA"],
                    ["요즘 볼만한거 있어?","MEDIA"],
                    ["영화나 예능 추천","MEDIA"],
                    ["재밌는 드라마 보여줘","MEDIA"],
                    ["신과 함께 줄거리 좀 알려줘","MEDIA"],
                    ["고등랩퍼 다시보기 좀","MEDIA"],
                    ["재밌는 영상 하이라이트만 보여줘","MEDIA"]]

test_data = [["쭈꾸미 맛집 좀 찾아줘","FOOD"],
                   ["매콤한 떡볶이 먹고싶다","FOOD"],
                   ["강남 씨지비 조조 영화 스케줄표 좀","MEDIA"],
                   ["효리네 민박 보고싶엉","MEDIA"]]

In [10]:
train_x, train_y = list(zip(*train_data))
print(train_x)
print(train_y)

('배고프다 밥줘', '뭐 먹을만한거 없냐', '맛집 추천', '이 근처 맛있는 음식점 좀', '밥줘', '뭐 먹지?', '삼겹살 먹고싶어', '영화 보고싶다', '요즘 볼만한거 있어?', '영화나 예능 추천', '재밌는 드라마 보여줘', '신과 함께 줄거리 좀 알려줘', '고등랩퍼 다시보기 좀', '재밌는 영상 하이라이트만 보여줘')
('FOOD', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'FOOD', 'MEDIA', 'MEDIA', 'MEDIA', 'MEDIA', 'MEDIA', 'MEDIA', 'MEDIA')


In [11]:
# train_x 데이터를 형태소 단위로 분리
train_x = [kor_tagger.morphs(x) for x in train_x]
print(train_x)

[['배고프', '다', '밥', '주', '어'], ['뭐', '먹', '을', '만하', 'ㄴ', '거', '없', '냐'], ['맛', '집', '추천'], ['이', '근처', '맛있', '는', '음식', '점', '좀'], ['밥', '주', '어'], ['뭐', '먹', '지', '?'], ['삼겹살', '먹', '고', '싶', '어'], ['영화', '보', '고', '싶', '다'], ['요즘', '볼만', '하', 'ㄴ', '거', '있', '어', '?'], ['영화', '나', '예능', '추천'], ['재밌', '는', '드라마', '보여주', '어'], ['신', '과', '함께', '줄거리', '좀', '알려주', '어'], ['고등', '랩', '푸', '어', '다시', '보', '기', '좀'], ['재밌', '는', '영상', '하이라이트', '만', '보여주', '어']]


In [12]:
# word-index dictionary
word2idx = {'<unk>':0}

for x in train_x:
    for token in x:
        if word2idx.get(token) == None:
            word2idx[token] = len(word2idx)
print(word2idx)
len(word2idx)

{'<unk>': 0, '배고프': 1, '다': 2, '밥': 3, '주': 4, '어': 5, '뭐': 6, '먹': 7, '을': 8, '만하': 9, 'ㄴ': 10, '거': 11, '없': 12, '냐': 13, '맛': 14, '집': 15, '추천': 16, '이': 17, '근처': 18, '맛있': 19, '는': 20, '음식': 21, '점': 22, '좀': 23, '지': 24, '?': 25, '삼겹살': 26, '고': 27, '싶': 28, '영화': 29, '보': 30, '요즘': 31, '볼만': 32, '하': 33, '있': 34, '나': 35, '예능': 36, '재밌': 37, '드라마': 38, '보여주': 39, '신': 40, '과': 41, '함께': 42, '줄거리': 43, '알려주': 44, '고등': 45, '랩': 46, '푸': 47, '다시': 48, '기': 49, '영상': 50, '하이라이트': 51, '만': 52}


53

In [13]:
# class-index dict
class2idx = {'FOOD':0, "MEDIA":1}
print(class2idx)

{'FOOD': 0, 'MEDIA': 1}


In [14]:
print(word2idx.get("패스트")) # train data에 존재하지 않는 형태소이기 때문에 none

None


In [15]:
# 문장과 word2idx이 주어진 문장 내의 단어를 onehot으로 변환
def make_bag_of_words(seq, word2idx):
    tensor = torch.zeros(len(word2idx))
    for w in seq:
        idx = word2idx.get(w)
        
        # 형태소 매칭 dictionary에 단어가 존재하는 경우
        if idx != None:
            tensor[idx] += 1. # onehot
        # 형태소 매칭 dictionary에 단어가 존재하지 않는 경우
        else:
            idx = word2idx['<unk>'] # 0
            tensor[idx] += 1.
        
    return tensor

In [16]:
# 각 문장 데이터에 대해 onehot 변환
train_x = torch.cat([make_bag_of_words(x, word2idx).to(device).view(1, -1) for x in train_x])
print(train_x.size())

torch.Size([14, 53])


In [17]:
# 각 문장 카테고리에 대해 변환
train_y = torch.cat([torch.LongTensor([class2idx[y]]).to(device) for y in train_y])
print(train_y.size())

torch.Size([14])


In [18]:
# 문장이 input으로 들어오면 해당 문장의 카테고리를 예측하는 classifier
class BOWClassifier(nn.Module):
    def __init__(self, vocab_size, output_size):
        super(BOWClassifier, self).__init__()
        
        self.linear = nn.Linear(vocab_size, output_size)
        
    def forward(self, x):
        return self.linear(x)

In [19]:
# training 과정
n_epochs = 100
lr = 0.1

model = BOWClassifier(len(word2idx), len(class2idx)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

In [20]:
for epoch in range(n_epochs):
    model.zero_grad()
    outputs = model(train_x)
    loss = criterion(outputs, train_y)
    
    if epoch % 10 == 0:
        print(loss.item())
        
    loss.backward()
    optimizer.step()

0.7054248452186584
0.5429993271827698
0.4370329678058624
0.36316439509391785
0.3090343177318573
0.267861932516098
0.23562932014465332
0.20980529487133026
0.1887173354625702
0.17121779918670654


In [21]:
idx2class = {v:k for k, v in class2idx.items()}
print(idx2class)

{0: 'FOOD', 1: 'MEDIA'}


In [22]:
print(test_data)

[['쭈꾸미 맛집 좀 찾아줘', 'FOOD'], ['매콤한 떡볶이 먹고싶다', 'FOOD'], ['강남 씨지비 조조 영화 스케줄표 좀', 'MEDIA'], ['효리네 민박 보고싶엉', 'MEDIA']]


In [27]:
# testing 과정
for test in test_data:
    x = kor_tagger.morphs(test[0]) # x에 대해 형태소 분리
    x = make_bag_of_words(x, word2idx).view(1, -1).to(device)
    
    outputs = model(x)
    outputs, idx = torch.max(outputs, 1)
    
    print(outputs, idx)
    
    print("inputs: %s" % test[0])
    print("prediction: %s" % idx2class[idx.item()])
    print("truth:%s" % test[1])
    print("\n")

tensor([ 0.6404]) tensor([ 0])
inputs: 쭈꾸미 맛집 좀 찾아줘
prediction: FOOD
truth:FOOD


tensor([ 0.2854]) tensor([ 0])
inputs: 매콤한 떡볶이 먹고싶다
prediction: FOOD
truth:FOOD


tensor([ 0.9578]) tensor([ 1])
inputs: 강남 씨지비 조조 영화 스케줄표 좀
prediction: MEDIA
truth:MEDIA


tensor([ 0.6815]) tensor([ 1])
inputs: 효리네 민박 보고싶엉
prediction: MEDIA
truth:MEDIA


